# <strong>Firewalls</strong>
Firewalls are security features for networks designed to accept or reject connections to certain websites and ports. There are two types of firewalls: <strong>Stateless</strong> and <strong>Stateful</strong> firewalls.

<figure><center><img src="resources/firewall/firewall_graphic.jpg" style="width: 85%; height: 85%;"></img></center><em><figcaption>Credit: <a href="https://www.elegantthemes.com/blog/wordpress/what-is-a-firewall-and-which-type-is-right-for-you">Elegant Themes</a></figcaption></em></figure>

#### <strong>Stateless Firewalls</strong>
Stateless firewalls use rules to accept or block connections based on the source, destination, and other predefined criteria, without tracking the connection's state. The <u>state</u> of a connection refers to its status based on the packet sent. More about states will be covered when discussing stateful firewalls. Stateless firewalls tend to be quicker because they use predefined rules to filter information. Since they do not track states, they can be faster and less resource-intensive, making them effective for handling heavy traffic loads. Typically, small companies opt for stateless firewalls due to their affordability. <a href="https://www.fortinet.com/resources/cyberglossary/stateful-vs-stateless-firewall">Because there is usually less incoming traffic than with a large enterprise, there may also be fewer threats.</a>

#### <strong>Stateful Firewalls</strong>
Stateful firewalls are more intelligent because they can track and monitor active connections. There are three types of states recognized by ```iptables```, the tool you will use to configure firewalls in this lab:

|       State       | Description|
|:-----------------:|--------------------------------------|
| ```NEW```         | A new connection. Has not been seen in both directions (incoming and outgoing). |
| ```ESTABLISHED``` | A previously-seen connection. Has been seen at one point going in either direction. |
| ```RELATED```     | A different connection, but has been previously associated with a prior connection, <br>like FTP (File Transfer Protocol) or ICMP (Internet Control Message Protocol) errors. <br>These errors occur when the destination tells the source that a packet couldn't be <br>forwarded/delivered. |

Stateful firewalls are better at blocking a wider range of attacks, as many attacks can exploit a connection's state and sequences. These firewalls maintain a <strong>state table</strong>, where connections that are new, established, or related are tracked and updated as connections are made. These tables track the destination/source IP addresses and ports, as well as the connection's state.

For this lab, we are focusing on <u>stateful</u> firewalls.

<strong>This lab will cover four topics, and you will learn the following:</strong>

1. Basic Unix Tools for Networking
2. Introduction to ```iptables```
3. Basic ```iptables``` Rules
4. A Large-Scale Application of ```iptables```

In [1]:
# Setting up the lab.
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript
from IPython.core.magic import register_line_magic
import os
# For accessing the nodes:
import subprocess
# For the stopexp command:
import re
import shlex

# When true, it will not auto-save at each step.
runAllSteps = False

###### Used for saving notebooks. ######
import threading
# Threading required in case steps are progressed too quickly.
save_lock = threading.Lock()

# The save function itself.
def save_notebook():
    with save_lock:
        result = subprocess.run('su - USERNAME_GOES_HERE -c "/project/USERNAME_GOES_HERE/notebooks/resources/save.py firewalls"', shell=True, capture_output=True, text=True)

# Creating a thread to save the notebook.
def trigger_save(question, response, answer=""):
    save_thread = threading.Thread(target=save_notebook)
    save_thread.start()
    
    command = (f'su - USERNAME_GOES_HERE -c "/home/.education/grader.py F {question} {response}"' 
               if answer == "" 
               else f'su - USERNAME_GOES_HERE -c "/home/.education/grader.py F {question} {response} \\"{answer}\\""')
    
    result = subprocess.run(command, shell=True, capture_output=True, text=True)

###### Used for loading notebooks. ######
import queue

load_lock = threading.Lock()
result_queue = queue.Queue()

# This variable is used to warn students if they haven't loaded a save yet.
def warn_student():
    if (os.path.exists("/project/USERNAME_GOES_HERE/notebooks/saves/.firewalls_warning")):
        os.remove("/project/USERNAME_GOES_HERE/notebooks/saves/.firewalls_warning")
        return True

def load_notebook():
    with load_lock:
        result = subprocess.run('su - USERNAME_GOES_HERE -c "/project/USERNAME_GOES_HERE/notebooks/resources/load.py firewalls"', shell=True, capture_output=True, text=True)
        result_queue.put(result)  # Put the result in the queue.

# Creating a thread to load the notebook.
def trigger_load():
    load_thread = threading.Thread(target=load_notebook)
    load_thread.start()
    load_thread.join()  # Wait for the thread to complete before adding result to the queue.
    return result_queue.get()  # Get the result from the queue.

###### Used for generating tables in Section 3. ######
def generate_table(step):
    # Create an "offset". Take the step number, subtract the offset, then that is the number of rows.
    offset = 8
    explanations = ['Add this rule to the ”filters” <strong>t</strong>able.', '<strong>A</strong>ppend to the OUTPUT chain.', 'Any packets that are <strong>o</strong>utbound from this interface will be affected.', 'All TCP <strong>p</strong>rotocols.',
                    'From the <strong>s</strong>ource IP, which is your network.', 'To the <strong>d</strong>estination, Google.', 'To this <strong>d</strong>estination <strong>port</strong>.', '<strong>M</strong>atching a state.', 'With the <strong>state</strong> being a NEW state.', 'Will be <strong>j</strong>umped (DROPPED).']

    final_rule = ""
    
    # Construct the start of the table.
    table = "<table class=\"center\"><tr><th style=\"text-align: center;\"><strong>Rule</strong></th><th style=\"text-align: center;\"><strong>Description</strong></th></tr>"

    # Create the sudo iptables row.
    table += "<tr><td style=\"font-family: 'Courier New', Courier, monospace;\">sudo iptables</td><td class=\"text-align: center;\">Calls the <span style=\"font-family: 'Courier New', Courier, monospace;\">iptables</span> command.</td></tr>"
    
    # Get the total number of rows.
    rows = step - offset

    # Create a loop, and start generating the rows.
    # This is being reversed so that the student's previous answers appear at the top of the table.
    for offset in reversed(range(0, rows)):
        # Get the student's response.
        result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "cat /home/.checker/responses/step_' + str(step - offset) + '_answer.txt"', shell=True, capture_output=True, text=True)

        # Take the result from the statement above.
        answers = (result.stdout).split()
    
        # Split every other space. So that all parameters and arguments are linked together.
        answers = [" ".join(answers[i:i+2]) for i in range(0, len(answers), 2)]

        # For the multiple answers...
        for answer in answers:
            # Perform some condition checks to see which rule is getting added to the table.
            if (answer.startswith("-t")):
                table += "<tr><td style=\"font-family: 'Courier New', Courier, monospace;\">" + answer + "</td><td>" + explanations[0] + "</td></tr>"

            elif (answer.startswith("-A")):
                table += "<tr><td style=\"font-family: 'Courier New', Courier, monospace;\">" + answer + "</td><td>" + explanations[1] + "</td></tr>"

            elif (answer.startswith("-o")):
                table += "<tr><td style=\"font-family: 'Courier New', Courier, monospace;\">" + answer + "</td><td>" + explanations[2] + "</td></tr>"

            elif (answer.startswith("-p")):
                table += "<tr><td style=\"font-family: 'Courier New', Courier, monospace;\">" + answer + "</td><td>" + explanations[3] + "</td></tr>"

            elif (answer.startswith("-s")):
                table += "<tr><td style=\"font-family: 'Courier New', Courier, monospace;\">" + answer + "</td><td>" + explanations[4] + "</td></tr>"

            elif (answer.startswith("-d")):
                table += "<tr><td style=\"font-family: 'Courier New', Courier, monospace;\">" + answer + "</td><td>" + explanations[5] + "</td></tr>"

            elif (answer.startswith("--dport")):
                table += "<tr><td style=\"font-family: 'Courier New', Courier, monospace;\">" + answer + "</td><td>" + explanations[6] + "</td></tr>"

            elif (answer.startswith("-m")):
                table += "<tr><td style=\"font-family: 'Courier New', Courier, monospace;\">" + answer + "</td><td>" + explanations[7] + "</td></tr>"

            elif (answer.startswith("--state")):
                table += "<tr><td style=\"font-family: 'Courier New', Courier, monospace;\">" + answer + "</td><td>" + explanations[8] + "</td></tr>"

            elif (answer.startswith("-j")):
                table += "<tr><td style=\"font-family: 'Courier New', Courier, monospace;\">" + answer + "</td><td>" + explanations[9] + "</td></tr>"

            final_rule += answer + " "

    # Table is complete. Closing it.
    table += "</table><br>"

    # The full rule so far:
    table += "<span><strong>Your rule:</strong> <span style=\"font-family: 'Courier New', Courier, monospace;\">sudo iptables " + final_rule + "</span></span>"

    # Finally, returning it.
    return table

### Step 0: Starting the Lab

Click the button to begin creating the experiment.

<strong>Note:</strong> If your buttons are not displaying, click on the <img width='20px' height='20px' style='margin-left: 1px;' src='resources/fast_forward.png'> icon at the top of your notebook to render all widgets.

In [2]:
# Click the button below to start the experiment.
import time

def check_autosave():
    # Check if the student has an autosave.
    if os.path.exists("/project/USERNAME_GOES_HERE/notebooks/saves/USERNAME_GOES_HERE_firewalls.tar.gz"):
        # Create a hidden file that will serve as a "boolean" for later.
        subprocess.run("touch /project/USERNAME_GOES_HERE/notebooks/saves/.firewalls_warning", shell=True)

def startlab(button):
    # Defining the lab name.
    labname = "firewalls"

    # Writing the information to an empty field below the button.
    with output0:
        # Fixes a strange error that happens only occasionally.
        os.chdir("/project/USERNAME_GOES_HERE/notebooks")
        output0.clear_output()

        # First, checking if the materialization exists. May have been stopped by a previous lab.
        materialPattern = "real." + labname + "jup.USERNAME_GOES_HERE"

        # Listing the materializations to find if there's an existing one for this lab.
        checkMaterial = os.popen('su - USERNAME_GOES_HERE -c "mrg list materializations"').read()
        regex = re.compile(materialPattern)
        # Getting the matches:
        match = regex.search(checkMaterial)

        if match:
            display(HTML("<span style='color: orange;'>An existing activation for this lab already exists. </span><span>You might have run another \
            lab without stopping this one. Attaching the existing activation...</span>"))

            # Detaching it and re-attaching.
            subprocess.run('su - USERNAME_GOES_HERE -c "mrg xdc detach xdc.USERNAME_GOES_HERE"', shell=True, check=True)
            subprocess.run('su - USERNAME_GOES_HERE -c "mrg xdc attach xdc.USERNAME_GOES_HERE real.' + labname + 'jup.USERNAME_GOES_HERE"', capture_output=True, text=True, shell=True, check=True)
            display(HTML("<span>Re-running the installation... </span><span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

            # Check if the student has an autosave.
            check_autosave()
            
            # Re-running the installation script.
            subprocess.run('su - USERNAME_GOES_HERE -c "bash /home/runlab ' + labname + 'jup"', capture_output=True, text=True, shell=True, check=True)
            output0.clear_output()
            display(HTML("<newline><span style='color: green;'><strong>Your lab has been re-installed. </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook.</span>"))
        
        else:
            display(HTML("<span>No existing activations are found.</span>"))

            # Second, start the lab.
            display(HTML("<span>Starting the " + labname + " lab. This will take a few minutes to process. Please wait.</span> \
            <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            try:
                startexp = subprocess.run('su - USERNAME_GOES_HERE -c "bash /home/startexp ' + labname + 'jup"', capture_output=True, text=True, shell=True, check=True)
            except:
                output0.clear_output()
                display(HTML("<span style='color: red;'>There was an error starting your experiment. Make sure your password is written inside of ~/pass.txt, and try again.</span>"))
                return
            
            print(startexp.stdout, flush=True)
            output0.clear_output()
            display(HTML("<span>Done. Result:</span>"))
            print(startexp.stdout, flush=True)

            # Another lab is already attached to the XDC.
            if ("XDC already attached" in startexp.stdout):
                existingLab = re.search(r"real.(.*).USERNAME_GOES_HERE", startexp.stdout).group(1)

                # Shouldn't happen.
                if labname == existingLab:
                    display(HTML("<span style='color: red;'>Your lab was already started. </span><span>Please continue to the next step.</span>"))

                # Detaching the existing lab, then attaching the current one.
                else:
                    display(HTML("<span style='color: orange;'>Warning: You did not stop your previous experiment. </span><span>Please stop your experiments \
                    before starting a new one. Detaching the " + existingLab + " experiment.</span>"))
                    subprocess.run('su - USERNAME_GOES_HERE -c "mrg xdc detach xdc.USERNAME_GOES_HERE"', shell=True, check=True)
                    display(HTML("<span>Attaching the current lab.</span>"))
                    subprocess.run('su - USERNAME_GOES_HERE -c "mrg xdc attach xdc ' + materialPattern + '"', shell=True, check=True)
    
            # Third, get the lab materials onto the node.
            display(HTML("<span>Allocating lab resources onto the node. <u>Please wait a little longer...</u></span>"))

            # Gives the notebook a couple seconds so that it will recognize the node(s).
            time.sleep(2)

            # Move the resources over.
            runlab = subprocess.run(
                'su - USERNAME_GOES_HERE -c "bash /home/runlab ' + labname + 'jup"',
                capture_output=True, text=True, shell=True
            )

            # Check if the student has an autosave.
            check_autosave()

            # Complete. Inform the student.
            display(HTML("<newline><span style='color: green;'><strong>Setup complete. You may begin the lab! </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook. Your lab will be active for one week.</span>"))


# Creating the button.
startButton = widgets.Button(description="Start Lab")

# Creating an output area.
output0 = widgets.Output()

# Run the command on click.
startButton.on_click(startlab)

# Display the output.
display(startButton, output0)

Button(description='Start Lab', style=ButtonStyle())

Output()

<hr>

If you previously stopped your lab, you may restore your progress below by clicking "Load Lab". <u>You do not have to load your lab if you signed out, closed your notebook, or exited your node(s) or XDC by using ```exit```.</u>

In [3]:
# Click the button below to load your lab.
def loadlab(b):
    with output0_2:
        output0_2.clear_output()
        display(HTML("<span>Searching for an existing lab in your notebook...</span>"))

    if (os.path.exists("/project/USERNAME_GOES_HERE/notebooks/saves/USERNAME_GOES_HERE_firewalls.tar.gz")):
        with output0_2:
            output0_2.clear_output()
            display(HTML("<span>Loading your lab...</span> \
                <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            result = trigger_load()
            if (result.returncode == 0):
                output0_2.clear_output()
                display(HTML("<span style='color: green;'>Your lab has been successfully loaded. Please click on the <img width='20px' height='20px' style='margin-left: 1px;' src='resources/fast_forward.png'> icon at the top of your notebook to reflect your changes.</span>"))
            elif (result.returncode == 2):
                output0_2.clear_output()
                display(HTML("<span style='color: red;'>The server node is inaccessible. Please start your lab. If you have already started it, wait a minute and try again.</span>"))
            else:
                output0_2.clear_output()
                display(HTML("<span style='color: red;'>An error occurred while loading your lab.</span>"))

# Creating the button.
loadButton = widgets.Button(description="Load Lab")

# Creating an output area.
output0_2 = widgets.Output()

# Run the command on click.
loadButton.on_click(loadlab)

# Display the output.
display(loadButton, output0_2)

Button(description='Load Lab', style=ButtonStyle())

Output()

## <strong>Topic 1: Basic Unix Tools for Networking</strong>

Before diving into firewalls, it's important to learn what tools are available to you for examining network connections. You will be provided a few tools that can assist you with checking the work that you make throughout the lab.

<span style="color: red;"><strong><img src="resources/alert.png" style="width: 12px"> Important:</strong></span> 
- The firewall lab does NOT have a ```firewall``` node. There are two separate nodes for this lab, and you will be setting up different firewall rules on the ```server``` and ```client``` nodes. When you access your lab, use ```ssh server``` and ```ssh client```. <strong>SSH'ing to a ```firewall``` node will not work!</strong>
- For this lab, you can SSH into the ```server``` and ```client``` without having to type ```exit``` to go between your XDC each time.

Throughout Topic 1, you may test these commands in the ```server``` node.

### Step 1: ```nmap``` - "<u>N</u>etwork <u>Map</u>ping"

```nmap``` is a useful tool that shows what ports that a website has access to your device. When using ```nmap```, you may call a website as an argument to see what ports that it uses on your machine. ```nmap``` has many additional uses, but for the purpose of this notebook, it will only be used to display the ports used by connections.

If you're interested in learning more about the uses of ```nmap```, you may check out some examples provided by the developers <a href="https://nmap.org/book/man-examples.html">here</a>.

<strong>Your task</strong>: Type a command below that uses ```nmap```, which displays the ports that <span style="color: blue;">yahoo.com</span> uses on your device. Do this in the ```server``` node.

In [4]:
# Click the button below to check your work.
step1Complete = False

# Function to check if the student's answer was correct.
def step_1():
    # Important variables that must be accessed outside of this function.
    global step1Complete, result

    # Loading, in case the check is slow.
    with output1:
        output1.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput1.value == ""):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step1Complete = False

    # Next, check to make sure that the wget command is used.
    pattern = r'^nmap'
    if (not re.match(pattern, userInput1.value)):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: red;'>Your answer does not use nmap.</span>"))
            step1Complete = False

    else:
        user_input = userInput1.value
        escaped_user_input = user_input.replace('"', '\\"').replace("'", "'\\''")

        # Construct the SSH command for testing the student's response.
        test_response = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "/home/.checker/section_1.py 1 '{escaped_user_input}'" """
        result = subprocess.run(test_response, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "echo '{escaped_user_input}' > /home/.checker/responses/step_1_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 1):
            output1.clear_output()
            with output1:
                display(HTML("<span style='color: green;'>Your command is correct!</span>"))
                step1Complete = True
        
        elif (result.returncode == 0):
            output1.clear_output()
            with output1:
                display(HTML("<span style='color: red;'>The command that you provided is incorrect.</span>"))
                step1Complete = False

        elif (result.returncode == 2):
            output1.clear_output()
            with output1:
                display(HTML("<span style='color: red;'>An error occurred. Please contact your professor or TA.</span>"))
                step1Complete = False

        
def check_step_1(b):
    if (warn_student()):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_1()

    # Auto-save.
    if (not runAllSteps):
        trigger_save("1", result.returncode, userInput1.value)

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading1 = widgets.Output()
display(loading1)
with loading1:
    loading1.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput1 = widgets.Text(
    placeholder='Type your nmap command here',
    description='Command:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "cat /home/.checker/responses/step_1_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
# Trim the newline.
userInput1.value = result.stdout[:-1]

# After the student's response was loaded, clear the output.
loading1.clear_output()

# Creating the button.
button = widgets.Button(description="Check Command")

# Creating an output area.
output1 = widgets.Output()

# Run the command on click.
button.on_click(check_step_1)

# Display the output.
display(userInput1, button, output1)

Output()

Text(value='nmap    yahoo.com ', description='Command:', layout=Layout(width='90%'), placeholder='Type your nm…

Button(description='Check Command', style=ButtonStyle())

Output()

### Step 2: ```ifconfig``` - "<u>I</u>nter<u>f</u>ace <u>Config</u>uration"

```ifconfig``` is used to view and configure the network interfaces on your device. Network interfaces can be physical hardware components such as Wi-Fi adapters and Ethernet connections that send and receive data over the internet. Each type of connection, whether it's a Wi-Fi adapter, Ethernet connection, or even a virtual network interface, is treated as a separate interface by the operating system. 

For example, using a Wi-Fi dongle on a device that already has an internal network adapter will result in two different interfaces being displayed by ```ifconfig```. 

<strong>Your task</strong>: Use ```ifconfig``` in the ```server``` node. In the field below, type the name of the network interface whose IP address is ```10.0.1.1```.

In [5]:
# Click the button below to check your work.
step2Complete = False

# Function to check if the student's answer was correct.
def step_2():
    # Important variables that must be accessed outside of this function.
    global step2Complete, result

    # Loading, in case the check is slow.
    with output2:
        output2.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput2.value == ""):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step2Complete = False

    else:
        user_input = userInput2.value
        escaped_user_input = user_input.replace('"', '\\"').replace("'", "'\\''")

        # Construct the SSH command for testing the student's response.
        test_response = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "/home/.checker/section_1.py 2 '{escaped_user_input}'" """
        result = subprocess.run(test_response, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "echo '{escaped_user_input}' > /home/.checker/responses/step_2_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 1):
            output2.clear_output()
            with output2:
                display(HTML("<span style='color: green;'>Your command is correct!</span>"))
                step2Complete = True
        
        elif (result.returncode == 0):
            output2.clear_output()
            with output2:
                display(HTML("<span style='color: red;'>The command that you provided is incorrect.</span>"))
                step2Complete = False

        elif (result.returncode == 2):
            output2.clear_output()
            with output2:
                display(HTML("<span style='color: red;'>An error occurred. Please contact your professor or TA.</span>"))
                step2Complete = False

        
def check_step_2(b):
    if (warn_student()):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_2()

    # Auto-save.
    if (not runAllSteps):
        trigger_save("2", result.returncode, userInput2.value)

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading2 = widgets.Output()
display(loading2)
with loading2:
    loading2.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput2 = widgets.Text(
    placeholder='Type your network interface here',
    description='Interface:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "cat /home/.checker/responses/step_2_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
# Trim the newline.
userInput2.value = result.stdout[:-1]

# After the student's response was loaded, clear the output.
loading2.clear_output()

# Creating the button.
button = widgets.Button(description="Check Interface")

# Creating an output area.
output2 = widgets.Output()

# Run the command on click.
button.on_click(check_step_2)

# Display the output.
display(userInput2, button, output2)

Output()

Text(value='eth1', description='Interface:', layout=Layout(width='90%'), placeholder='Type your network interf…

Button(description='Check Interface', style=ButtonStyle())

Output()

### Step 3: ```telnet``` - "<u>Tel</u>e-<u>Net</u>work"

```telnet``` is a cleartext remote terminal protocol. On the surface, ```telnet``` is very simple: the user issues commands over a TCP socket, and the server replies with the results of those commands and waits for more input. ```telnet``` is one of the simplest and oldest network protocols still in use. Due to its cleartext nature and low-level access to the system, ```telnet``` is incredibly insecure. 

In the past, it was common for system administrators to log in as root using telnet on a network connection that could be easily sniffed by any sufficiently prepared attacker. This lack of encryption means that usernames, passwords, and other sensitive data are transmitted in plaintext, making it vulnerable to interception.

Telnetting to a suspected open port is still one of the fastest ways to see if a service is available or reachable. Here is an example of a ```telnet``` request.

```
$ telnet yahoo.com 80

Trying 66.94.234.13...
Connected to yahoo.com.
Escape character is '^]'.
GET /
...

<html><head> ...[web page data] ...
</body>
</html>

Connection closed by foreign host.
```

A small breakdown of what's being typed:
- ```telnet yahoo.com 80``` calls Yahoo with an HTTP request, which is port 80.
- ```GET /``` gets the index page of Yahoo, which is ```http://www.yahoo.com/```. Anything appears after the end of the URL (the ```/```) appears in the HTTP response. If you wanted to get a specific webpage, like ```/watch.html```, this would be in the GET request that you would use. 

Telnetnetting to an IP and port (see above) should return a "connected" message if it is possible to connect to a running server.

<strong>Your task</strong>: Use ```telnet``` to retrieve <span style="color: blue;">https://scse.d.umn.edu/majors-minors</span>. Note the use of ```https``` within the URL. An ```https``` request uses port ```443```. Once you have a working connection, type the ```telnet``` command that you used, as well as the ```HTTP``` argument that you used to retrieve the webpage.

In [ ]:
# Click the button below to check your work.
step3Complete = False

# Function to check if the student's answer was correct.
def step_3():
    # Important variables that must be accessed outside of this function.
    global step3Complete, result

    # Loading, in case the check is slow.
    with output3:
        output3.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput3_1.value == "" or userInput3_2.value == ""):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>One of your input fields is empty.</span>"))
            step3Complete = False

    # Next, check to make sure that the telnet command is used.
    elif (not re.match(r'^telnet', userInput3_1.value)):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>Your answer does not start with (or use) telnet.</span>"))
            step3Complete = False

    else:
        user_input = userInput3_1.value + "\\n" + userInput3_2.value
        escaped_user_input = user_input.replace("'", "'\\''")

        # Construct the SSH command for testing the student's response.
        test_response = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "/home/.checker/section_1.py 3 '{escaped_user_input}'" """
        result = subprocess.run(test_response, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "echo '{escaped_user_input}' > /home/.checker/responses/step_3_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 1):
            output3.clear_output()
            with output3:
                display(HTML("<span style='color: green;'>Your commands are correct!</span>"))
                step3Complete = True
        
        elif (result.returncode == 0):
            output3.clear_output()
            with output3:
                display(HTML("<span style='color: red;'>A command that you provided is incorrect.</span>"))
                step3Complete = False

        elif (result.returncode == 2):
            output3.clear_output()
            with output3:
                display(HTML("<span style='color: red;'>An error occurred. Please contact your professor or TA.</span>"))
                step3Complete = False

        
def check_step_3(b):
    if (warn_student()):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_3()

    # Auto-save.
    if (not runAllSteps):
        trigger_save("3", result.returncode, f"{userInput3_1.value} {userInput3_2.value}")

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading3 = widgets.Output()
display(loading3)
with loading3:
    loading3.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating two text areas.
userInput3_1 = widgets.Text(
    placeholder='Type your telnet command here',
    description='Command:',
    layout=widgets.Layout(width='90%')
)

userInput3_2 = widgets.Text(
    placeholder='Type your telnet response here',
    description='Request:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "cat /home/.checker/responses/step_3_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)

userInput3_1.value = ""
userInput3_2.value = ""

if (result.stdout != ""):
    # Split the results.
    result_stdout = re.sub(r'\n', '', result.stdout)
    split_result = (result_stdout).split("\\n")
    
    # Assign the values and trim the newline.
    userInput3_1.value = split_result[0]
    userInput3_2.value = split_result[1]

    if (userInput3_2.value[:-2] == "\n"):
        userInput3_2.value = userInput3_2.value[:-2]

# After the student's response was loaded, clear the output.
loading3.clear_output()

# Creating the button.
button = widgets.Button(description="Check Commands")

# Creating an output area.
output3 = widgets.Output()

# Run the command on click.
button.on_click(check_step_3)

# Display the output.
display(userInput3_1, userInput3_2, button, output3)

Output()

Text(value='telnet scse.d.umn.edu 443', description='Command:', layout=Layout(width='90%'), placeholder='Type …

Text(value='GET /majors-minors', description='Response:', layout=Layout(width='90%'), placeholder='Type your t…

Button(description='Check Commands', style=ButtonStyle())

Output()

### Step 4: ```nc``` - "<u>N</u>et<u>c</u>at"

```nc``` is a Unix utility for creating and using TCP and UDP sockets. In a very simplified way, ```nc``` is like a telnet client and server without any built in protocol or terminal emulation. Another way of putting it is that netcat is the bare essentials for creating a TCP or UDP socket and client, with hooks for using standard in and standard out for IO.

Netcat can be used for several purposes, such as: Sending messages to computers, debugging connection issues, scanning for open ports on a remote host (```nc -zv hostname 20-80``` scans between ports 20-80 on the specified hostname), transferring files, creating a backdoor (which bypasses security systems), connecting to a remote shell, and a lot more.

<strong>Your task</strong>: On the ```server``` node, set up a server that listens for TCP protocols, and listen on port 10000. On the ```client``` node, set up a connection to the ```server``` node that sends TCP packets to ```server``` on port 10000. Once you have set this up, try sending a message through ```client``` to see if it will be displayed on ```server```.

There are text entries below, where you will type in the commands that you used to set up ```server``` and ```client```. Once you have a working "chatroom", type the two commands below that worked. They will be tested.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tips:</strong></span> 
- When using the ```nc``` command, you will need to use ```-l``` which means "listen", and ```-p``` which accepts a port number afterwards. When using ```nc``` to connect to another computer, you can simply just type ```nc <name> <port>``` to send messages to a specified address/name that's listening for requests.
- Try opening two terminals to simultaneously access both nodes.
- Step 18 has an example of how to do this with step, but with UDP packets. <u>Do not use UDP packets for this step.</u>
- Attempting to call your ```nc``` command on ```client``` <strong>before</strong> ```server``` listens on port 10000 might display a "Connection Refused" error. Ensure that you begin listening on port 10000 first before calling a command on ```client```. 

In [7]:
# Click the button below to check your work.
step4Complete = False

# Function to check if the student's answer was correct.
def step_4():
    # Important variables that must be accessed outside of this function.
    global step4Complete, result

    # Loading, in case the check is slow.
    with output4:
        output4.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput4_1.value == "" or userInput4_2.value == ""):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'>One of your input fields is empty.</span>"))
            step4Complete = False

    # Next, check to make sure that the telnet command is used.
    elif (not re.match(r'^nc', userInput4_1.value) or not re.match(r'^nc', userInput4_2.value)):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'>Your answer does not use telnet.</span>"))
            step4Complete = False

    else:
        user_input = userInput4_1.value + "\\n" + userInput4_2.value
        escaped_user_input = user_input.replace("\\", "\\\\").replace("'", "'\\''")

        # Construct the SSH command for testing the student's response.
        test_response = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "/home/.checker/section_1.py 4 '{escaped_user_input}'" """
        result = subprocess.run(test_response, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "echo '{escaped_user_input}' > /home/.checker/responses/step_4_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 1):
            output4.clear_output()
            with output4:
                display(HTML("<span style='color: green;'>Your commands are correct!</span>"))
                step4Complete = True
        
        elif (result.returncode == 0):
            output4.clear_output()
            with output4:
                display(HTML("<span style='color: red;'>The command that you provided is incorrect.</span>"))
                step4Complete = False

        elif (result.returncode == 2):
            output4.clear_output()
            with output4:
                display(HTML("<span style='color: red;'>An error occurred. Please contact your professor or TA.</span>"))
                step4Complete = False

        
def check_step_4(b):
    if (warn_student()):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_4()

    # Auto-save.
    if (not runAllSteps):
        trigger_save("4", result.returncode, f"{userInput4_1.value} {userInput4_2.value}")

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading4 = widgets.Output()
display(loading4)
with loading4:
    loading4.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating two text areas.
userInput4_1 = widgets.Text(
    placeholder='Type your server\'s command here',
    description='Server:',
    layout=widgets.Layout(width='90%')
)

userInput4_2 = widgets.Text(
    placeholder='Type your client\'s command here',
    description='Client:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "cat /home/.checker/responses/step_4_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)

userInput4_1.value = ""
userInput4_2.value = ""

if (result.stdout != ""):
    # Split the results.
    split_result = re.split(r'(?:\\n|\n)', result.stdout)
    
    # Trim the newline.
    userInput4_1.value = split_result[0]
    userInput4_2.value = split_result[1]

# After the student's response was loaded, clear the output.
loading4.clear_output()

# Creating the button.
button = widgets.Button(description="Check Commands")

# Creating an output area.
output4 = widgets.Output()

# Run the command on click.
button.on_click(check_step_4)

# Display the output.
display(userInput4_1, userInput4_2, button, output4)

Output()

Text(value='nc -l -p 10000', description='Server:', layout=Layout(width='90%'), placeholder="Type your server'…

Text(value='nc server 10000', description='Client:', layout=Layout(width='90%'), placeholder="Type your client…

Button(description='Check Commands', style=ButtonStyle())

Output()

## <strong>Topic 2: Introduction to ```iptables```</strong>

In the previous section, you have explored some tools that you may find useful with creating firewall rules in this lab. Now, you're going to look at the next Unix tool, which is ```iptables```. You will be spending the remaining part of the lab learning about ```iptables```. In the next topic, you will start to learn about the parameters that ```iptables``` accepts in your rules. In the final section, you will be given a list of scenarios and apply your own rules.

This section will walk you through the basic interface of ```iptables```.

### Step 5: Viewing ```iptable``` Rules

The ```iptables``` tool has been pre-installed for you on both nodes. However, you will need to navigate into ```server``` for the remainder of this topic.

Type ```sudo iptables -L``` into ```server```. ```sudo``` is required for this tool, since this tool lets you control the network flow across the computer. The ```-L``` argument stands for "list". Typing this command is going to list all of the rules that are currently in your rule table.

These are the only three policies at the moment:
```
-P INPUT ACCEPT
-P FORWARD ACCEPT
-P OUTPUT ACCEPT
```

Before providing a breakdown of these three policies, you are going to read the term <strong>"chain"</strong> when working with ```iptables```. A chain is a set of rules. Hence, you're "linking" the rules together to form a chain.

Here's a small breakdown of the default rules so far:
- The ```-P``` argument means "policy". This parameter is used with three built-in chains: ```INPUT```, ```FORWARD```, or ```OUTPUT```. There are more built-in chains than these, but these are the most common ones when using this argument.
  - Note, ```-p``` stands for "protocol". These two should not be mixed up when you begin to make your own rules. You can use either ```-P```, or to be more precise, ```--policy``` also works.
- ```INPUT```, ```FORWARD```, and ```OUTPUT``` are the incoming, forwarded, and outgoing packets.
- ```ACCEPT``` is the policy of the chain. It tells your system what to do with the chain that preceeds it.
  - <strong>All ```iptables``` rules end with ```ACCEPT```, ```DROP```, or ```REJECT```.</strong>
    - ```ACCEPT``` allows the connection.
    - ```DROP``` disallows the connection, but sends no response to whoever is accessing it.
    - ```REJECT``` disallows the connection, but sends a response to whoever is accessing it.


After reading this breakdown, this should make the default rules clear in ```iptables```. This means that all incoming, forwarded, and outgoing packets are accepted. Nothing is being blocked.

<strong>Your task</strong>: Add the following rule to the table. You will learn about what it does in the next two steps.

```sudo iptables -A OUTPUT -p tcp --dport 80 -j DROP```

You may view this rule with ```sudo iptables -S```, where ```-S``` is short for "show".

In [8]:
# Click the button below to check your work.
step5Complete = False

# Function to check the permissions.
def step_5():
    # Important variables that must be accessed outside of this function.
    global step5Complete, result

    with output5:
        output5.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # This subprocess statement is a little different. Need to initiate environment variables at the same time when running the command.
    # This SSH command needs to be tweaked a little so that environment variables work across SSH.
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "bash -l -c \'/home/.checker/section_2.py 5 NA\'"', shell=True, capture_output=True, text=True)

    if (result.returncode == 1):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step5Complete = True

    elif (result.returncode == 0):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>The command that was provided to you cannot be found as a rule. Try again.</span>"))
            step5Complete = False

    elif (result.returncode == 2):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>There was an error with checking your step. Please contact your professor or TA for assistance.</span>"))
            step5Complete = False
    
def check_step_5(b):
    if (warn_student()):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_5()

    # Auto-save.
    if (not runAllSteps):
        trigger_save("5", result.returncode)

# Creating the button.
button = widgets.Button(description="Check Rule")

# Creating an output area.
output5 = widgets.Output()

# Run the command on click.
button.on_click(check_step_5)

# Display the output.
display(button, output5)

Button(description='Check Rule', style=ButtonStyle())

Output()

### Step 6: Identifying the Chain and Jump

With this rule added, it's important to understand what this rule has done.

Here is a breakdown of the rule that you just added:
- ```sudo iptables``` calls the ```iptables``` command.
- ```-A OUTPUT``` <strong>appends</strong> the rules that you made to the built-in ```OUTPUT``` chain.
- ```-p tcp``` indicates that the rule affects TCP <strong>packets</strong>.
- ```--dport 80``` indicates that the rule affects the <strong>destination port</strong>
- ```-j DROP``` stands for <strong>jumps</strong>. However, it's easier to interpret this as "everything that matches this rule will be DROPPED".

Note that one of the default rules is ```-P OUTPUT ACCEPT```, which indicates that one of your policies is to "accept all output".

The reason why ```-A``` is being used is because you are taking the currently existing ```-P OUTPUT ACCEPT``` rule, but you're appending a rule that states, "Accept all output, BUT... (your rule here)".

Using the two blanks below, type in the <strong>chain</strong> and <strong>jump</strong> of the rule that you wrote.

In [9]:
# Click the button below to check your work.
step6Complete = False

# Function to check if the student's answer was correct.
def step_6():
    # Important variables that must be accessed outside of this function.
    global step6Complete, result

    # Loading, in case the check is slow.
    with output6:
        output6.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput6_1.value == "" or userInput6_2.value == ""):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'>One of your input fields is empty.</span>"))
            step6Complete = False

    else:
        user_input = userInput6_1.value + "\n" + userInput6_2.value
        escaped_user_input = user_input.replace('"', '\\"').replace("'", "'\\''")

        # Construct the SSH command for testing the student's response.
        test_response = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "/home/.checker/section_2.py 6 '{escaped_user_input}'" """
        result = subprocess.run(test_response, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "echo '{escaped_user_input}' > /home/.checker/responses/step_6_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 1):
            output6.clear_output()
            with output6:
                display(HTML("<span style='color: green;'>Correct!</span>"))
                step6Complete = True
        
        elif (result.returncode == 0):
            output6.clear_output()
            with output6:
                display(HTML("<span style='color: red;'>Check your responses. One (or both) of them is incorrect.</span>"))
                step6Complete = False

        elif (result.returncode == 2):
            output6.clear_output()
            with output6:
                display(HTML("<span style='color: red;'>An error occurred. Please contact your professor or TA.</span>"))
                step6Complete = False

        
def check_step_6(b):
    if (warn_student()):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_6()

    # Auto-save.
    if (not runAllSteps):
        trigger_save("6", result.returncode, f"{userInput6_1.value} {userInput6_2.value}")

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading6 = widgets.Output()
display(loading6)
with loading6:
    loading6.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating two text areas.
userInput6_1 = widgets.Text(
    placeholder='Type your "chain" answer here',
    description='Chain:',
    layout=widgets.Layout(width='90%')
)

userInput6_2 = widgets.Text(
    placeholder='Type your "jump" answer here',
    description='Jump:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "cat /home/.checker/responses/step_6_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)

userInput6_1.value = ""
userInput6_2.value = ""

if (result.stdout != ""):
    # Split the results.
    split_result = (result.stdout).split("\n")
    
    # Trim the newline.
    userInput6_1.value = split_result[0]
    userInput6_2.value = split_result[1]

# After the student's response was loaded, clear the output.
loading6.clear_output()

# Creating the button.
button = widgets.Button(description="Check Commands")

# Creating an output area.
output6 = widgets.Output()

# Run the command on click.
button.on_click(check_step_6)

# Display the output.
display(userInput6_1, userInput6_2, button, output6)

Output()

Text(value='OUTPUT', description='Chain:', layout=Layout(width='90%'), placeholder='Type your "chain" answer h…

Text(value='DROP', description='Jump:', layout=Layout(width='90%'), placeholder='Type your "jump" answer here'…

Button(description='Check Commands', style=ButtonStyle())

Output()

### Step 7: Testing Your Rule

Type a command that will attempt to access a website through port 80. You will need to type a command that attempts to access a website.

Some accepted commands:
- ```telnet```
- ```nc```
- ```curl``` is also acceptable, but wasn't demonstrated in Topic 1.
  - If you use ```curl```, you'll need to test it through port 80. Calling ```curl``` without specifying a port will not work.

In [10]:
# Click the button below to check your work.
step7Complete = False

# Function to check if the student's answer was correct.
def step_7():
    # Important variables that must be accessed outside of this function.
    global step7Complete, result

    # Loading, in case the check is slow.
    with output7:
        output7.clear_output()
        display(HTML("<span>This step will take a little longer to compute. Please wait... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput7.value == ""):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step7Complete = False

    else:
        user_input = userInput7.value
        escaped_user_input = shlex.quote(user_input)
        
        # Construct the SSH command for testing the student's response.
        # This SSH command needs to be tweaked a little so that environment variables work across SSH.
        ssh_command = (
            f"ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "
            f"\"/home/.checker/section_2.py 7 '{escaped_user_input}'\""
        )
        
        # Run the command
        result = subprocess.run(f'ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "/home/.checker/section_2.py 7 {escaped_user_input}"', shell=True, capture_output=True, text=True)

        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "echo '{escaped_user_input}' > /home/.checker/responses/step_7_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 1):
            output7.clear_output()
            with output7:
                display(HTML("<span style='color: green;'>Correct!</span>"))
                step7Complete = True
        
        elif (result.returncode == 0):
            output7.clear_output()
            with output7:
                display(HTML("<span style='color: red;'>Your command doesn't work. Make sure you are using port 80 and a valid website. Check if you may have extra firewall rules.</span>"))
                step7Complete = False

        elif (result.returncode == 2):
            output7.clear_output()
            with output7:
                display(HTML("<span style='color: red;'>An error occurred. Please contact your professor or TA.</span>"))
                step7Complete = False

        
def check_step_7(b):
    if (warn_student()):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_7()

    # Auto-save.
    if (not runAllSteps):
        trigger_save("7", result.returncode, userInput7.value)

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading7 = widgets.Output()
display(loading7)
with loading7:
    loading7.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput7 = widgets.Text(
    placeholder='Type a command here to test the rule',
    description='Command:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "cat /home/.checker/responses/step_7_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
# Trim the newline.
userInput7.value = result.stdout[:-1]

# After the student's response was loaded, clear the output.
loading7.clear_output()

# Creating the button.
button = widgets.Button(description="Check Command")

# Creating an output area.
output7 = widgets.Output()

# Run the command on click.
button.on_click(check_step_7)

# Display the output.
display(userInput7, button, output7)

Output()

Text(value='nc -zv google.com 80', description='Command:', layout=Layout(width='90%'), placeholder='Type a com…

Button(description='Check Command', style=ButtonStyle())

Output()

### Step 8: Deleting a Rule

To delete a rule, you can use ```sudo iptables -D [rule]```. Clearly, the ```-D``` stands for delete. Previously, you used ```-A``` to append a rule.

Using your intuition, delete the rule that you made in Step 5.

In [ ]:
# Click the button below to check your work.
step8Complete = False

# Function to check the permissions.
def step_8():
    # Important variables that must be accessed outside of this function.
    global step8Complete, result

    with output8:
        output8.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # This subprocess statement is a little different. Need to initiate environment variables at the same time when running the command.
    # This SSH command needs to be tweaked a little so that environment variables work across SSH.
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "bash -l -c \'/home/.checker/section_2.py 8 NA\'"', shell=True, capture_output=True, text=True)

    if (result.returncode == 1):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: green;'>Correct! Additionally, using \"sudo iptables -F\" wipes ALL rules, leaving the default rules left. The -F stands for \"flush\".</span>"))
            step8Complete = True

    elif (result.returncode == 0):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>Your rule doesn't appear to be deleted from iptables.</span>"))
            step8Complete = False

    elif (result.returncode == 2):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>There was an error with checking your step. Please contact your professor or TA for assistance.</span>"))
            step8Complete = False
    
def check_step_8(b):
    if (warn_student()):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_8()

    # Auto-save.
    if (not runAllSteps):
        trigger_save("8", result.returncode)

# Creating the button.
button = widgets.Button(description="Check Rule")

# Creating an output area.
output8 = widgets.Output()

# Run the command on click.
button.on_click(check_step_8)

# Display the output.
display(button, output8)

Button(description='Check Rule', style=ButtonStyle())

Output()

## <strong>Topic 3: Basic ```iptables``` Rules </strong>

This topic is now going to teach you some ```iptables``` rules. You have already seen a little bit of syntax in the previous step. Now, you're going to look more in-depth with the syntax.

```iptables``` allows you to create very broad commands, like the one you have seen above, which drops all TCP packets on port 80. Additionally, it allows you to create very specific commands, like the one you're about to see in this topic. The following steps are going to start you with a very basic example, and you are going to work your way up to very specific parameters.

<u>As you complete each step in this topic</u>, a table will be generated so that you can see how your rule is being progressively created. The table will provide a "translation", which is describing how your rules are being defined.

If you are feeling lost with ```iptables``` syntax, these are two helpful documents that you may look into while you go through the final two topics:
- <a href="https://linux.die.net/man/8/iptables">```iptables``` Documentation</a>
- <a href="https://www.informit.com/articles/article.aspx?p=421057&seqNum=4">Linux Firewalls, 3rd Edition - Iptables Chapter</a>

### Step 9: Tables

This is the start of the ```iptables``` rule that you'll be producing throughout the topic:

```sudo iptables```

<strong>You are going to build up a complex ```iptables``` rule one step at a time. This does not need to be typed into your node(s).</strong> The entire construction of this rule will take place within your notebook.

There are multiple tables that you can add rules to in ```iptables```. For the lab, you're just adding tables to the "filters" table. This is the default table in ```iptables```. If you don't specify a table in ```iptables```, it will automatically append your rule to the "filters" table. The remaining tables are as follows <a href="https://gist.github.com/nerdalert/a1687ae4da1cc44a437d">(source)</a>:
- NAT table - Iptable's NAT table has the following built-in chains.
- Mangle table - Iptables's Mangle table is for specialized packet alteration.
- Raw table - Iptable's Raw table is for configuration excemptions.

You are going to append to the currently existing command. This is the current template of your rule so far:

```sudo iptables [your answer here]```

<strong>Your task</strong>: Type a parameter that adds your rule to the "filters" table. Use the ```-t``` argument for "table". For your answer, just type in the ```[your answer here]``` portion of the rule.

In [12]:
# Click the button below to check your work.
step9Complete = False

# Function to check if the student's answer was correct.
def step_9():
    # Important variables that must be accessed outside of this function.
    global step9Complete, result

    # Loading, in case the check is slow.
    with output9:
        output9.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput9.value == ""):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step9Complete = False

    else:
        user_input = userInput9.value
        escaped_user_input = user_input.replace('"', '\\"').replace("'", "'\\''")

        # Construct the SSH command for testing the student's response.
        # This SSH command needs to be tweaked a little so that environment variables work across SSH.
        test_response = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "/home/.checker/section_3.py 9 \\"{escaped_user_input}\\"" """
        result = subprocess.run(test_response, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "echo '{escaped_user_input}' > /home/.checker/responses/step_9_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 1):
            output9.clear_output()
            with output9:
                display(HTML("<span style='color: green;'>Correct! Here are what your rules do so far. Take note of the <strong>bolded</strong> letters to help remember what the parameters stand for.</span>"))
                # Creating the table for students.
                display(HTML(generate_table(9)))
                step9Complete = True
        
        elif (result.returncode == 0):
            output9.clear_output()
            with output9:
                display(HTML("<span style='color: red;'>Your rule appears to be invalid. Please re-read the directions and try again.</span>"))
                step9Complete = False

        elif (result.returncode == 2):
            output9.clear_output()
            with output9:
                display(HTML("<span style='color: red;'>An error occurred. Please contact your professor or TA.</span>"))
                step9Complete = False

        
def check_step_9(b):
    if (warn_student()):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_9()

    # Auto-save.
    if (not runAllSteps):
        trigger_save("9", result.returncode, userInput9.value)

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading9 = widgets.Output()
display(loading9)
with loading9:
    loading9.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput9 = widgets.Text(
    placeholder='Type your rule here',
    description='Rule:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "cat /home/.checker/responses/step_9_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
# Trim the newline.
userInput9.value = result.stdout[:-1]

# After the student's response was loaded, clear the output.
loading9.clear_output()

# Creating the button.
button = widgets.Button(description="Check Rule")

# Creating an output area.
output9 = widgets.Output()

# Run the command on click.
button.on_click(check_step_9)

# Display the output.
display(userInput9, button, output9)

Output()

Text(value='-t   filters', description='Rule:', layout=Layout(width='90%'), placeholder='Type your rule here')

Button(description='Check Rule', style=ButtonStyle())

Output()

### Step 10: Network Interface

Now, you can add a network interface that you want your rule to be applied to. You can do this by using ```-i``` for "inbound", or ```-o``` for "outbound". This means that any packets that are sent to/from this network interface will be affected by the rule. From Step 2, you already found what the network interface is attached to ```server```.

Here is the work so far that you've produced:

```sudo iptables [step 9] [your next answer]```

<strong>Your task</strong>: Type in the following parameters which will match the following rules:

- Use the network interface for outbound packets that you found in Step 2.
- Append your rule to the pre-defined OUTPUT chain.
  - ```iptables``` rules are CASE SENSITIVE.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Note:</strong></span> You have two rules to type for this step. More will be required in future steps. <u>Order does not matter</u> when typing these parameters.

In [13]:
# Click the button below to check your work.
step10Complete = False

# Function to check if the student's answer was correct.
def step_10():
    # Important variables that must be accessed outside of this function.
    global step10Complete, result

    # Loading, in case the check is slow.
    with output10:
        output10.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput10.value == ""):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step10Complete = False

    else:
        user_input = userInput10.value
        escaped_user_input = user_input.replace('"', '\\"').replace("'", "'\\''")

        # Construct the SSH command for testing the student's response.
        # This SSH command needs to be tweaked a little so that environment variables work across SSH.
        test_response = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "/home/.checker/section_3.py 10 \\"{escaped_user_input}\\"" """
        result = subprocess.run(test_response, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "echo '{escaped_user_input}' > /home/.checker/responses/step_10_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 1):
            output10.clear_output()
            with output10:
                display(HTML("<span style='color: green;'>Correct! Here are what your rules do so far.</span>"))
                # Creating the table for students.
                display(HTML(generate_table(10)))
                step10Complete = True
        
        elif (result.returncode == 0):
            output10.clear_output()
            with output10:
                display(HTML("<span style='color: red;'>Your rule appears to be invalid. Please re-read the directions and try again.</span>"))
                step10Complete = False

        elif (result.returncode == 2):
            output10.clear_output()
            with output10:
                display(HTML("<span style='color: red;'>An error occurred. Please contact your professor or TA.</span>"))
                step10Complete = False

        
def check_step_10(b):
    if (warn_student()):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_10()

    # Auto-save.
    if (not runAllSteps):
        trigger_save("10", result.returncode, userInput10.value)

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading10 = widgets.Output()
display(loading10)
with loading10:
    loading10.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput10 = widgets.Text(
    placeholder='Type your rule here',
    description='Rule:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "cat /home/.checker/responses/step_10_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
# Trim the newline.
userInput10.value = result.stdout[:-1]

# After the student's response was loaded, clear the output.
loading10.clear_output()

# Creating the button.
button = widgets.Button(description="Check Command")

# Creating an output area.
output10 = widgets.Output()

# Run the command on click.
button.on_click(check_step_10)

# Display the output.
display(userInput10, button, output10)

Output()

Text(value='-o eth1   -A OUTPUT', description='Rule:', layout=Layout(width='90%'), placeholder='Type your rule…

Button(description='Check Command', style=ButtonStyle())

Output()

### Step 11: States and Protocols

Next, you're going to add a state to the rule. This is done by using the ```-m``` and ```--state``` command. These two parameters are used in conjunction with each other. ```-m``` stands for "match", and ```--state``` is clearly indicating the state to match to.

Recall states from the beginning of the notebook. There are ```NEW```, ```ESTABLISHED```, and ```RELATED``` states. This is the template for creating a state rule: ```-m state --state STATE_NAME```

A breakdown:
- ```-m``` means match.
- ```state``` means you're going to match a state.
- ```--state``` indicates that the next parameter is going to be a state.
- ```STATE_NAME``` is either ```NEW```, ```ESTABLISHED```, or ```RELATED```.

<u>Note here that the only parameter you need to change is STATE_NAME.</u>

Additionally, you read above that ```-p``` means "protocol". Two common protocols are UDP and TCP protocols. If you haven't taken a networking course, you may read about the two protocols <a href="https://stackoverflow.com/questions/5970383/difference-between-tcp-and-udp">here</a>. Essentially, <strong>UDP</strong> is <strong>U</strong>nreliable, but fast. <strong>TCP</strong> is <strong>T</strong>rustworthy, but slow.

Here's the rule so far:

```sudo iptables [step 9] [step 10] [your next answer]```

Clearly, you should see how the remaining steps in this topic will work. Additionally, your answer is being printed as you work along.

<strong>Your task</strong>: Type in the following parameters which will match the following rules:

- Create a rule that matches the NEW state.
- Detect TCP packets.
  - Hint: Look back at Step 6.

In [14]:
# Click the button below to check your work.
step11Complete = False

# Function to check if the student's answer was correct.
def step_11():
    # Important variables that must be accessed outside of this function.
    global step11Complete, result

    # Loading, in case the check is slow.
    with output11:
        output11.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput11.value == ""):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step11Complete = False

    else:
        user_input = userInput11.value
        escaped_user_input = user_input.replace('"', '\\"').replace("'", "'\\''")

        # Construct the SSH command for testing the student's response.
        # This SSH command needs to be tweaked a little so that environment variables work across SSH.
        test_response = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "/home/.checker/section_3.py 11 \\"{escaped_user_input}\\"" """
        result = subprocess.run(test_response, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "echo '{escaped_user_input}' > /home/.checker/responses/step_11_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 1):
            output11.clear_output()
            with output11:
                display(HTML("<span style='color: green;'>Correct! Here are what your rules do so far.</span>"))
                # Creating the table for students.
                display(HTML(generate_table(11)))
                step11Complete = True
        
        elif (result.returncode == 0):
            output11.clear_output()
            with output11:
                display(HTML("<span style='color: red;'>Your rule appears to be invalid. Please re-read the directions and try again.</span>"))
                step11Complete = False

        elif (result.returncode == 2):
            output11.clear_output()
            with output11:
                display(HTML("<span style='color: red;'>An error occurred. Please contact your professor or TA.</span>"))
                step11Complete = False

        
def check_step_11(b):
    if (warn_student()):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_11()

    # Auto-save.
    if (not runAllSteps):
        trigger_save("11", result.returncode, userInput11.value)

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading11 = widgets.Output()
display(loading11)
with loading11:
    loading11.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput11 = widgets.Text(
    placeholder='Type your rule here',
    description='Rule:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "cat /home/.checker/responses/step_11_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
# Trim the newline.
userInput11.value = result.stdout[:-1]

# After the student's response was loaded, clear the output.
loading11.clear_output()

# Creating the button.
button = widgets.Button(description="Check Rule")

# Creating an output area.
output11 = widgets.Output()

# Run the command on click.
button.on_click(check_step_11)

# Display the output.
display(userInput11, button, output11)

Output()

Text(value='-m state --state NEW -p tcp', description='Rule:', layout=Layout(width='90%'), placeholder='Type y…

Button(description='Check Rule', style=ButtonStyle())

Output()

### Step 12: Sources and Destinations

For this step, you're going to apply a source and destination to this rule. These two parameters are fairly straightforward. You will use ```-s``` for the source IP address, and ```-d``` for the destination IP address (OR the domain). Additionally, you can use ```--sport``` or ```--dport``` for the source/destination ports of the rule.

<strong>Your task</strong>: Type in the following parameters which will match the following rules:

- Use the source of the network interface that you found in Step 2. You will use this IP address for the source IP address.
- Use the destination of ```google.com```.
  - <u>Instead of using ```-d google.com```, which is a valid entry, you are required to find an IP address of Google for this step.</u> Try using one of the methods above in Section 1 to find this.
- Add a rule that uses Google's destination port of 1234.

In [15]:
# Click the button below to check your work.
step12Complete = False

# Function to check if the student's answer was correct.
def step_12():
    # Important variables that must be accessed outside of this function.
    global step12Complete, result

    # Loading, in case the check is slow.
    with output12:
        output12.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput12.value == ""):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step12Complete = False

    else:
        user_input = userInput12.value
        escaped_user_input = user_input.replace('"', '\\"').replace("'", "'\\''")

        # Construct the SSH command for testing the student's response.
        # This SSH command needs to be tweaked a little so that environment variables work across SSH.
        test_response = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "/home/.checker/section_3.py 12 \\"{escaped_user_input}\\"" """
        result = subprocess.run(test_response, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "echo '{escaped_user_input}' > /home/.checker/responses/step_12_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 1):
            output12.clear_output()
            with output12:
                display(HTML("<span style='color: green;'>Correct! Here are what your rules do so far.</span>"))
                # Creating the table for students.
                display(HTML(generate_table(12)))
                step12Complete = True
        
        elif (result.returncode == 0):
            output12.clear_output()
            with output12:
                display(HTML("<span style='color: red;'>Your rule appears to be invalid. Please re-read the directions and try again.</span>"))
                step12Complete = False

        elif (result.returncode == 2):
            output12.clear_output()
            with output12:
                display(HTML("<span style='color: red;'>An error occurred. Please contact your professor or TA.</span>"))
                step12Complete = False

        elif (result.returncode == 3):
            output12.clear_output()
            with output12:
                display(HTML("<span style='color: red;'>Your rule is almost correct. The IP of google.com appears incorrect. Try again.</span>"))
                step12Complete = False

        
def check_step_12(b):
    if (warn_student()):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_12()

    # Auto-save.
    if (not runAllSteps):
        trigger_save("12", result.returncode, userInput12.value)

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading12 = widgets.Output()
display(loading12)
with loading12:
    loading12.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput12 = widgets.Text(
    placeholder='Type your rule here',
    description='Rule:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "cat /home/.checker/responses/step_12_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
# Trim the newline.
userInput12.value = result.stdout[:-1]

# After the student's response was loaded, clear the output.
loading12.clear_output()

# Creating the button.
button = widgets.Button(description="Check Rule")

# Creating an output area.
output12 = widgets.Output()

# Run the command on click.
button.on_click(check_step_12)

# Display the output.
display(userInput12, button, output12)

Output()

Text(value='-s 10.0.1.1 -d 142.251.40.46 --dport 1234', description='Rule:', layout=Layout(width='90%'), place…

Button(description='Check Rule', style=ButtonStyle())

Output()

### Step 13: Jumps

Finally, you are going to do the jump. This indicates what you are going to do with packets that match all of these rules that you composed so far. You have already seen how a jump works. Use the ```-j``` command. Recall that all ```iptables``` rules will end with a jump.

<strong>Your task</strong>: Drop all connections that match the rule that you have been constructing in this topic.

In [16]:
# Click the button below to check your work.
step13Complete = False

# Function to check if the student's answer was correct.
def step_13():
    # Important variables that must be accessed outside of this function.
    global step13Complete, result

    # Loading, in case the check is slow.
    with output13:
        output13.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput13.value == ""):
        output13.clear_output()
        with output13:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step13Complete = False

    else:
        user_input = userInput13.value
        escaped_user_input = user_input.replace('"', '\\"').replace("'", "'\\''")

        # Construct the SSH command for testing the student's response.
        # This SSH command needs to be tweaked a little so that environment variables work across SSH.
        test_response = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "/home/.checker/section_3.py 13 \\"{escaped_user_input}\\"" """
        result = subprocess.run(test_response, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "echo '{escaped_user_input}' > /home/.checker/responses/step_13_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 1):
            output13.clear_output()
            with output13:
                display(HTML("<span style='color: green;'>Correct! Here are what your rules do so far. <strong>Your final rule is displayed below.</strong></span>"))
                # Creating the table for students.
                display(HTML(generate_table(13)))
                step13Complete = True
        
        elif (result.returncode == 0):
            output13.clear_output()
            with output13:
                display(HTML("<span style='color: red;'>Your rule appears to be invalid. Please re-read the directions and try again. (Hint: Your answer is case-sensitive!)</span>"))
                step13Complete = False

        elif (result.returncode == 2):
            output13.clear_output()
            with output13:
                display(HTML("<span style='color: red;'>An error occurred. Please contact your professor or TA.</span>"))
                step13Complete = False

        
def check_step_13(b):
    if (warn_student()):
        output13.clear_output()
        with output13:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_13()

    # Auto-save.
    if (not runAllSteps):
        trigger_save("13", result.returncode, userInput13.value)

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading13 = widgets.Output()
display(loading13)
with loading13:
    loading13.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput13 = widgets.Text(
    placeholder='Type your rule here',
    description='Rule:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@server "cat /home/.checker/responses/step_13_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
# Trim the newline.
userInput13.value = result.stdout[:-1]

# After the student's response was loaded, clear the output.
loading13.clear_output()

# Creating the button.
button = widgets.Button(description="Check Rule")

# Creating an output area.
output13 = widgets.Output()

# Run the command on click.
button.on_click(check_step_13)

# Display the output.
display(userInput13, button, output13)

Output()

Text(value='-j DROP', description='Rule:', layout=Layout(width='90%'), placeholder='Type your rule here')

Button(description='Check Rule', style=ButtonStyle())

Output()

## <strong>Topic 4: A Large-Scale Application of ```iptables```</strong>

In this final topic, you are going to be given an instruction to block or accept certain connections. Using all of the knowledge you used in the previous topics, you are prepared to attempt to create your own rules.

Once again, as provided in the previous topic, you're free to use these two resources if you feel stuck with the syntax for ```iptables```:

- <a href="https://linux.die.net/man/8/iptables">```iptables``` Documentation</a>
- <a href="https://www.informit.com/articles/article.aspx?p=421057&seqNum=4">Linux Firewalls, 3rd Edition - Iptables Chapter</a>

Additionally, these are the following IP addresses that you will need to use for this section:

| Nodes:        | ```server```   | ```client```   |
|---------------|----------------|----------------|
| IP Addresses: | ```10.0.1.1``` | ```10.0.1.2``` |

#### <strong>Useful Information For This Section</strong>:

- Whenever you would like to reset your ```iptables``` rules, you may type ```sudo iptables -F```. The ```-F``` command means "flush", which will reset your table back to what it originally was. If you had already completed a step, your progress will not be lost after flushing the table.

- Additionally, <u>you will need to indicate a specific protocol in each of your steps</u>. In other words, all of your rules must have ```-p udp``` or ```-p tcp``` in them. By default, connections that are made between ```client``` and ```server``` are TCP connections. However, the last step will require a UDP connection.

- Use ```sudo iptables -S``` to view all current rules.

- You are strongly encouraged to look for keywords in each scenario. These are important keywords to look out for:

  - Inbound and outbound
  - TCP and UDP
  - ```server``` and ```client```

### Step 14: Inbound TCP Connections to the Apache Server

As you do the questions provided in this topic, you will be provided a "Check Rule" button. These checks will take time to complete, since they're testing for time-outs. Additionally, you are going to be told which node that these firewall rules will need to be placed on.

<strong>Scenario:</strong> On the ```server``` node, <strong>```DROP```</strong> inbound TCP connections to the Apache port from ```client```. This port is ```80```. Currently, this port is open to all connections to the ```server``` node.

<strong>A successful test appears as follows:</strong>
```
USERNAME_GOES_HERE@client:~$ telnet server 80
Trying 10.0.1.1...
^C
USERNAME_GOES_HERE@client:~$
```

Click the button below to check your work.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tips:</strong></span> 
- You will need to use ```client```'s IP address in your rule.
- <strong>All rules in this section require a ```-p``` argument. Look at the title for each step to know which protocol to use.</strong>
- Think carefully about when to use ```--dport``` or ```--sport```.

<span style="color: red;"><strong><img src="resources/alert.png" style="width: 12px"> Warning:</strong></span> You are now starting to apply your own firewall rules, which means that an incorrect firewall rule could potentially lock you out from your node. If you get locked out, you will have to end your lab (at the bottom of the notebook), then restart your lab by clicking "Start Lab" at the top of the notebook. Your progress is automatically saved at each step, so you do not need to worry about losing progress. If you restart your lab, make sure to reload your lab.

In [17]:
# Click the button below to check your work.
from subprocess import Popen, PIPE
step14Complete = False

# Function to check the permissions.
def step_14():
    # Important variables that must be accessed outside of this function.
    global step14Complete, result

    with output14:
        output14.clear_output()
        display(HTML("<span>Testing your firewall rule... <img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # This subprocess statement is a little different. Need to call a local script instead of a remote one.
    result = subprocess.run('su - USERNAME_GOES_HERE -c "/project/USERNAME_GOES_HERE/notebooks/resources/firewall/section_4.py 14"', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output14.clear_output()
        with output14:
            display(HTML("<span style='color: green;'>Correct! Your firewall rule worked correctly. If you have previously passed this step, this step will not fail. Feel free to remove this rule from your table.</span>"))
            step14Complete = True

    elif (result.returncode == 0):
        output14.clear_output()
        with output14:
            display(HTML("<span style='color: red;'>Your rule did not perform the given scenario correctly. Try again.</span>"))
            step14Complete = False

    elif (result.returncode == 3):
        output14.clear_output()
        with output14:
            display(HTML("<span style='color: red;'>The port number that's required for this step cannot be found in your rules. Try again.</span>"))
            step14Complete = False

    elif (result.returncode == 2):
        output14.clear_output()
        with output14:
            display(HTML("<span style='color: red;'>There was an error with checking your step. Please contact your professor or TA for assistance.</span>"))
            step14Complete = False
    
def check_step_14(b):
    if (warn_student()):
        output14.clear_output()
        with output14:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_14()

    # Auto-save.
    if (not runAllSteps):
        trigger_save("14", result.returncode)

# Creating the button.
button = widgets.Button(description="Check Rule")

# Creating an output area.
output14 = widgets.Output()

# Run the command on click.
button.on_click(check_step_14)

# Display the output.
display(button, output14)

Button(description='Check Rule', style=ButtonStyle())

Output()

### Step 15: Inbound TCP Connections to the MySQL Server

<strong>Scenario:</strong> On the ```server``` node, <strong>```DROP```</strong> all inbound TCP connections to the MySQL port from ```client```. This port is ```3306```. Currently, this port is open to all connections to the ```server``` node.

<strong>A successful test appears as follows:</strong>
```
USERNAME_GOES_HERE@client:~$ telnet server 3306
Trying 10.0.1.1...
^C
USERNAME_GOES_HERE@client:~$
```

Click the button below to check your work.

In [18]:
# Click the button below to check your work.
step15Complete = False

# Function to check the permissions.
def step_15():
    # Important variables that must be accessed outside of this function.
    global step15Complete, result

    with output15:
        output15.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # This subprocess statement is a little different. Need to call a local script instead of a remote one.
    result = subprocess.run('su - USERNAME_GOES_HERE -c "/project/USERNAME_GOES_HERE/notebooks/resources/firewall/section_4.py 15"', shell=True, capture_output=True, text=True)

    if (result.returncode == 1):
        output15.clear_output()
        with output15:
            display(HTML("<span style='color: green;'>Correct! Your firewall rule worked correctly. If you have previously passed this step, this step will not fail. Feel free to remove this rule from your table.</span>"))
            step15Complete = True

    elif (result.returncode == 0):
        output15.clear_output()
        with output15:
            display(HTML("<span style='color: red;'>Your rule did not perform the given scenario correctly. Try again.</span>"))
            step15Complete = False

    elif (result.returncode == 3):
        output15.clear_output()
        with output15:
            display(HTML("<span style='color: red;'>The port number that's required for this step cannot be found in your rules. Try again.</span>"))
            step15Complete = False

    elif (result.returncode == 2):
        output15.clear_output()
        with output15:
            display(HTML("<span style='color: red;'>There was an error with checking your step. Please contact your professor or TA for assistance.</span>"))
            step15Complete = False
    
def check_step_15(b):
    if (warn_student()):
        output15.clear_output()
        with output15:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_15()

    # Auto-save.
    if (not runAllSteps):
        trigger_save("15", result.returncode)

# Creating the button.
button = widgets.Button(description="Check Rule")

# Creating an output area.
output15 = widgets.Output()

# Run the command on click.
button.on_click(check_step_15)

# Display the output.
display(button, output15)

Button(description='Check Rule', style=ButtonStyle())

Output()

### Step 16: Outbound TCP Connections to a Specific HTTPS Request

<strong>Scenario:</strong> On the ```client``` node, <strong>```ACCEPT```</strong> outbound HTTPS connections to <span style="color: blue;">stackoverflow.com</span>, but <strong>```REJECT```</strong> everything else. The port for HTTPS requests is ```443```. Stack Overflow should be the only website that can be accessed through the ```client``` node.

<u>A destination IP address can work, but IP addresses may change.</u> Consider reviewing Topic 1 to view what tools you can use to find this IP. Instead, <u>the destination can be a URL</u>.

Domains use multiple IP addresses, depending on what the DNS server returns. If one fails, it will use the next available IP address until it returns successfully. The IP address that you use in the rule doesn't matter, as long as it's one of Stack Overflow's.

<strong>A successful test appears as follows:</strong>
```
USERNAME_GOES_HERE@client:~$ telnet stackoverflow.com 443
Trying XXX.XX.XX.X...
Trying XXX.XX.XXX.XXX...
Connected to stackoverflow.com.
Escape character is '^]'.
Connection closed by foreign host.
USERNAME_GOES_HERE@client:~$
```

Click the button below to check your work. 

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Note:</strong></span> In order for this test to pass, other random websites will be accessed. Other websites must not be accessible.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tip:</strong></span> <strong>Two rules are required for this step.</strong> Rules for ```iptables``` go in order. If you attempt to block all outgoing connections first, then allow Stack Overflow through the firewall, it will not work. You must allow Stack Overflow first, then block the remaining connections.

In [19]:
# Click the button below to check your work.
step16Complete = False

# Function to check the permissions.
def step_16():
    # Important variables that must be accessed outside of this function.
    global step16Complete, result

    with output16:
        output16.clear_output()
        display(HTML("<span>Please wait. This will take some extra time. Multiple websites are tested for this step... <img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # This subprocess statement is a little different. Need to call a local script instead of a remote one.
    result = subprocess.run('su - USERNAME_GOES_HERE -c "/project/USERNAME_GOES_HERE/notebooks/resources/firewall/section_4.py 16"', shell=True, capture_output=True, text=True)

    if (result.returncode == 1):
        output16.clear_output()
        with output16:
            display(HTML("<span style='color: green;'>Correct! Your firewall rule worked correctly. If you have previously passed this step, this step will not fail. Feel free to remove this rule from your table.</span>"))
            step16Complete = True

    elif (result.returncode == 0):
        output16.clear_output()
        with output16:
            display(HTML("<span style='color: red;'>Your rule did not perform the given scenario correctly. Try again.</span>"))
            step16Complete = False

    elif (result.returncode == 3):
        output16.clear_output()
        with output16:
            display(HTML("<span style='color: red;'>The port number that's required for this step cannot be found in your rules. Try again.</span>"))
            step16Complete = False

    elif (result.returncode == 2):
        output16.clear_output()
        with output16:
            display(HTML("<span style='color: red;'>There was an error with checking your step. Please contact your professor or TA for assistance.</span>"))
            step16Complete = False
    
def check_step_16(b):
    if (warn_student()):
        output16.clear_output()
        with output16:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_16()

    # Auto-save.
    if (not runAllSteps):
        trigger_save("16", result.returncode)

# Creating the button.
button = widgets.Button(description="Check Rule")

# Creating an output area.
output16 = widgets.Output()

# Run the command on click.
button.on_click(check_step_16)

# Display the output.
display(button, output16)

Button(description='Check Rule', style=ButtonStyle())

Output()

### Step 17: Outbound TCP OpenSSH Requests

<strong>Scenario:</strong> On the ```client``` node, <strong>```REJECT```</strong> all outbound SSH requests. OpenSSH uses port 22.

<strong>A successful test appears as follows:</strong>
```
USERNAME_GOES_HERE@client:~$ ssh server
ssh: connect to host server port 22: Connection refused
USERNAME_GOES_HERE@client:~$
```

Click the button below to check your work. 

<span style="color: red;"><strong><img src="resources/alert.png" style="width: 12px"> Warning:</strong></span> Double-check that you are going to deny any <strong>outbound</strong> SSH requests. If you accidentally deny any incoming SSH requests, you may lose access to your ```client``` node, and the lab must be restarted. <strong>Additionally</strong>, make sure that this rule is applied to the ```client``` node. If this rule is accidentally applied to the ```server``` node, the notebook will not save properly.

In [20]:
# Click the button below to check your work.
step17Complete = False

# Function to check the permissions.
def step_17():
    # Important variables that must be accessed outside of this function.
    global step17Complete, result

    with output17:
        output17.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # This subprocess statement is a little different. Need to call a local script instead of a remote one.
    result = subprocess.run('su - USERNAME_GOES_HERE -c "/project/USERNAME_GOES_HERE/notebooks/resources/firewall/section_4.py 17"', shell=True, capture_output=True, text=True)

    if (result.returncode == 1):
        output17.clear_output()
        with output17:
            display(HTML("<span style='color: green;'>Correct! Your firewall rule worked correctly. If you have previously passed this step, this step will not fail. Feel free to remove this rule from your table.</span>"))
            step17Complete = True

    elif (result.returncode == 0):
        output17.clear_output()
        with output17:
            display(HTML("<span style='color: red;'>Your rule did not perform the given scenario correctly. Try again.</span>"))
            step17Complete = False

    elif (result.returncode == 3):
        output17.clear_output()
        with output17:
            display(HTML("<span style='color: red;'>The port number that's required for this step cannot be found in your rules. Try again.</span>"))
            step17Complete = False

    elif (result.returncode == 2):
        output17.clear_output()
        with output17:
            display(HTML("<span style='color: red;'>There was an error with checking your step. Please contact your professor or TA for assistance.</span>"))
            step17Complete = False
    
def check_step_17(b):
    if (warn_student()):
        output17.clear_output()
        with output17:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_17()

    # Auto-save.
    if (not runAllSteps):
        trigger_save("17", result.returncode)

# Creating the button.
button = widgets.Button(description="Check Rule")

# Creating an output area.
output17 = widgets.Output()

# Run the command on click.
button.on_click(check_step_17)

# Display the output.
display(button, output17)

Button(description='Check Rule', style=ButtonStyle())

Output()

### Step 18: Inbound UDP Traffic on Ports 10000:10005

<strong>Scenario:</strong> On the ```server``` node, <strong>allow</strong> incoming UDP connections to be made on six ports from ```client```: ```10000```, ```10001```, ```10002```, ```10003```, ```10004```, and ```10005```. Additionally, <strong>```DROP```</strong> any other UDP packet that is sent from the ```client``` node. <u>You will apply these rules on ```server```.</u>

These six ports should be the only ports that ```client``` can send UDP packets to ```server``` on. <strong>You do not need to make six independent rules for the ports.</strong> You can just use ```10000:10005```. <u>If you do not use a port range like this, your step won't pass!</u>

<strong>A successful test appears as follows:</strong>

<u>Part 1:</u>
```
USERNAME_GOES_HERE@server:~$ nc -u -l -p 10000
Hello!

USERNAME_GOES_HERE@client:~$ nc -u server 10000
(Type Hello!)
```

<u>Part 2:</u>

```
USERNAME_GOES_HERE@server:~$ nc -u -l -p 10006


USERNAME_GOES_HERE@client:~$ nc -u server 10006
(Type Hello!)
```

Click the button below to check your work. 

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tips:</strong></span> 
- Like before, <strong>two rules on ```server``` are required for this step.</strong> One to accept the UDP packets through the specified ports from ```client```, and one to drop the remaining UDP packets from ```client``` on the remaining ports. Remember that order matters when adding ```iptables``` rules!
- When classifying the ports, use ```--dport``` instead of ```--sport```. When calling netcat from the ```client```, its connection is randomly assigned a port number. When packets are sent, the port that you classified in your argument is the port that the packets are being sent to. Thus, the port that you used in command would be the destination port.

<span style="color: red;"><strong><img src="resources/alert.png" style="width: 12px"> Warning:</strong></span> ```server``` relies on some UDP connections for ```iptables``` to run properly. Therefore, make sure to add the source IP address for ```client```, which is provided at the start of the topic. If you do not add a ```-s``` parameter, where the source is the ```client```, you might break ```iptables```!

In [21]:
# Click the button below to check your work.
step18Complete = False

# Function to check the permissions.
def step_18():
    # Important variables that must be accessed outside of this function.
    global step18Complete, result

    with output18:
        output18.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # This subprocess statement is a little different. Need to call a local script instead of a remote one.
    result = subprocess.run('su - USERNAME_GOES_HERE -c "/project/USERNAME_GOES_HERE/notebooks/resources/firewall/section_4.py 18"', shell=True, capture_output=True, text=True)

    if (result.returncode == 1):
        output18.clear_output()
        with output18:
            display(HTML("<span style='color: green;'>Correct! Your firewall rule worked correctly. If you have previously passed this step, this step will not fail. Feel free to remove this rule from your table.</span>"))
            step18Complete = True

    elif (result.returncode == 0):
        output18.clear_output()
        with output18:
            display(HTML("<span style='color: red;'>Your rule did not perform the given scenario correctly. Try again.</span>"))
            step18Complete = False

    elif (result.returncode == 3):
        output18.clear_output()
        with output18:
            display(HTML("<span style='color: red;'>The port number that's required for this step cannot be found in your rules. Try again.</span>"))
            step18Complete = False

    elif (result.returncode == 2):
        output18.clear_output()
        with output18:
            display(HTML("<span style='color: red;'>There was an error with checking your step. Please contact your professor or TA for assistance.</span>"))
            step18Complete = False
    
def check_step_18(b):
    if (warn_student()):
        output18.clear_output()
        with output18:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_18()

    # Auto-save.
    if (not runAllSteps):
        trigger_save("18", result.returncode)

# Creating the button.
button = widgets.Button(description="Check Rule")

# Creating an output area.
output18 = widgets.Output()

# Run the command on click.
button.on_click(check_step_18)

# Display the output.
display(button, output18)

Button(description='Check Rule', style=ButtonStyle())

Output()

## <strong>Grading</strong>

To check your overall grade, click on the button below.

In [22]:
# Click the button below to check your overall grade.
steps_to_check = [step_1, step_2, step_3, step_4, step_5, step_6, step_7, step_8, step_9, step_10, step_11, step_12, step_13, step_14, step_15, step_16, step_17, step_18]   

# Function to calculate grade after refreshing the cell
def calculate_grade(b):
    # To not auto-save at each step.
    global runAllSteps
    runAllSteps = True

    with gradeOutput:
        gradeOutput.clear_output()
        display(HTML("<span>Testing all steps. Please wait.</span> \
            <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # Required for checking the boolean values.
    for func in steps_to_check:
        func()  # Call each function in order.

    # Assuming steps are updated above this cell in some way
    steps = [step1Complete, step2Complete, step3Complete, step4Complete, step5Complete, step6Complete, step7Complete, step8Complete, step9Complete, step10Complete, step11Complete, step12Complete, step13Complete, step14Complete, step15Complete, step16Complete, step17Complete, step18Complete]
    output = ""
    stepsCorrect = 0
    numOfSteps = len(steps)

    for i in range(numOfSteps):
        if steps[i]:
            stepsCorrect += 1
            output += "<div style='color: green;'>Step " + str(i + 1) + " is complete.</div>"
        else:
            output += "<div style='color: red;'>Step " + str(i + 1) + " is incomplete.</div>"

    output += "<div style='color: black;'>You have " + str(stepsCorrect) + " out of " + str(numOfSteps) + " steps completed.</div>"

    with gradeOutput:
        gradeOutput.clear_output()
        display(HTML(output))

    # Enables auto-saving to happen at every step again.
    runAllSteps = False

# Create a button to refresh the cell and another to calculate grade.
grade_button = widgets.Button(description="Calculate Grade")

# Link buttons to functions.
grade_button.on_click(calculate_grade)

# Output area.
gradeOutput = widgets.Output()

# Display the buttons and output.
display(grade_button, gradeOutput)

Button(description='Calculate Grade', style=ButtonStyle())

Output()

### Stopping the Lab

Once you are done with the lab, click on the "Stop Lab" button below. <strong>This will delete your materialization, which will delete all of the lab's resources.</strong> Your progress is saved automatically in ```saves/``` within the sidebar of your XDC. You may load this lab in the future by clicking "Load Lab" at the top.

In [23]:
# Click the button below to stop the experiment.
def stoplab(button):
    # Check to make sure that the student wants to confirm ending the lab.
    if (confirm.value == False):
        with stop_output:
            stop_output.clear_output()
            display(HTML("<newline><span style='color: red;'>Please confirm that you wish to end the lab.</span>"))

    else:
        # Defining the lab name.
        labname = "firewalls"
    
        # Writing the information to an empty field below the button.
        with stop_output:
            stop_output.clear_output()
            
            display(HTML("<span>Stopping the " + labname + " lab. This will take a minute to process. Please wait.</span> \
                <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            stopexp = subprocess.run('su - USERNAME_GOES_HERE -c "bash /share/stopexp ' + labname + 'jup"', capture_output=True, text=True, shell=True)
            stop_output.clear_output()
            display(HTML("<span>Done. Result:</span>"))
            print(stopexp.stdout)
    
            display(HTML("<newline><span style='color: green;'><strong>Your lab has been ended.</strong></span>"))

# Creating the button.
stopButton = widgets.Button(description="Stop Lab")

# Create a confirmation check.
confirm = widgets.Checkbox(
    value=False,
    description='Confirm',
    disabled=False,
    indent=False
)

# Creating an output area.
stop_output = widgets.Output()

# Run the command on click.
stopButton.on_click(stoplab)

# Display the output.
display(confirm, stopButton, stop_output)

Checkbox(value=False, description='Confirm', indent=False)

Button(description='Stop Lab', style=ButtonStyle())

Output()